In [1]:
%load_ext watermark
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from myst_nb import glue

from plastockconf import name_zones, name_particles, name_frequentation, name_situation
from plastockconf import particle_groups, name_substrate, name_distance, table_css_styles

from plastock import add_table_to_page, capitalize_x_tick_labels, capitalize_x_and_y_axis_labels, capitalize_legend_components, attribute_summary

a_property =  {'color' : 'red'}
format_kwargs = dict(precision=2, thousands="'", decimal=",")
glue('blank_caption', " ", display=False)

def translate_describe(x, value_column, transpose: bool = False):
    described = x.to_dict()
    described.pop("count")
    described["moyenne"] = described.pop("mean")
    described["écart-type"] = described.pop("std")
    df = pd.DataFrame(described.items())
    df.set_index(0, inplace=True)
    df.rename(columns={1:value_column}, inplace=True)
    df.index.name = None
    
    if transpose:
        df = df.T
        
    return df
def translate_columns(data, columns, keys):
    
    for i, column in enumerate(columns):
        
        data[column] = data[column].apply(lambda x: keys[i][x])
    
    return data

def select_a_set_of_values(data, column_one, search_index, search_value, average_of_sample_formes):
    
    return data.loc[data[column_one] > average_of_sample_formes.loc[search_index, search_value]]

# thes samples were not completed
drop_these = ['VD_Cul_2', 'VD_Vid_13', 'VD_Vid_8', 'VS_Bou_12']


work_data = pd.read_csv("data/end_pipe/long_form_micro.csv")
work_data = work_data[~work_data["echantillon"].isin(drop_these)].copy()
work_data.rename(columns={'echantillon':'échantillon', 'frequentation':'fréquentation'}, inplace=True)
beach_data = pd.read_csv("data/end_pipe/asl_beaches.csv")

(micro_atts)=
# Microplastiques plage et position

La position fait référence à l'emplacement de l'échantillon de microplastique sur la plage. La ligne d'eau correspond aux échantillons prélevés très près de l'endroit où l'eau rencontre le rivage. La plage sèche fait référence aux endroits de la plage qui sont plus éloignés de la ligne d'eau et qui sont généralement secs.

__Résultats agrégés par forme, position et échantillon__

In [2]:
format_kwargs = dict(precision=2, thousands="'", decimal=",")

caption = 'Les données sont regroupées en trois groupes par échantillon: 1. fibres, 2. particules rigides, 3. particules souples.'
section = 'A'
rule = 'Les attributs dont la moyenne des résultats est supérieure à la moyenne du projet sont en rouge.'
page = 1
table_no =1

work_data_sample = work_data[work_data['échantillon'] == "74_Amp_2"].head()
work_data_sample['objet'] = work_data_sample.objet.apply(lambda x: particle_groups[x])
work_data_sample.set_index('objet', drop=True, inplace=True)
work_data_sample.index.name = None
work_data_sample = work_data_sample.style.set_table_styles(table_css_styles)
table_one = add_table_to_page(work_data_sample, table_no, caption, section, page, " ")
glue('tablea11', table_one, display=True)

,Plage,Échantillon,Orientation,Position,Substrat,Fréquentation,Situation,Distance,Compte
Fibre,Amphion,74_Amp_2,NNE,1,4,3,1,1,121
Particule rigide,Amphion,74_Amp_2,NNE,1,4,3,1,1,3
Particule souple,Amphion,74_Amp_2,NNE,1,4,3,1,1,12


## Nombre cumulé d'échantillons et particules par plage

In [3]:
work_data["particules"] = work_data["compte"]
sample_totals = work_data.groupby(["Plage","échantillon"], as_index=False).particules.sum()
df2 = sample_totals.copy()
value_column = "moyenne"

sample_averages = sample_totals.groupby("Plage", as_index=False).agg({"échantillon": "nunique", "particules":"mean"})
average_of_sample_totals = df2.particules.mean()
sample_averages.rename(columns={"particules":value_column}, inplace=True)
sample_averages.set_index("Plage", inplace=True, drop=True)
sample_averages.index.name = None

In [4]:
location_summary = work_data.groupby("Plage", as_index=False).agg({"échantillon": "nunique", "particules":"sum"})
location_summary["% échantillon"] = location_summary['échantillon']/location_summary['échantillon'].sum()
location_summary["% particules"] = location_summary.particules/location_summary.particules.sum()

df1 = location_summary.copy()
df1.set_index("Plage", inplace=True, drop=True)
df1.index.name = None

a_property =  {'color' : 'red'}
a_property2 = { 'color': 'dodgerblue'}

select_values = df1["% échantillon"] < df1["% particules"]

test_one = df1.loc[select_values].index
caption1 = "Nombre cumulé d'échantillons et le total par plage. Les plages où le proportion du comptage est supérieur au proportion des échantillons sont en rouge."
caption2 = 'Les plages dont la moyenne des résultats est supérieure à la moyenne du projet sont en BLEU.'

caption = f'{caption1} {caption2}'
df1["moyenne"] = df1.index.map(lambda x: sample_averages['moyenne'].loc[x])
select_values2 = df1["moyenne"] > 180
test_two = df1[select_values2].index

ttwo = df1.style.set_table_styles(table_css_styles).set_properties(subset=pd.IndexSlice[test_two, ["moyenne"]], **a_property2)
ttwo = ttwo.set_properties(subset = pd.IndexSlice[test_one,["% échantillon", "% particules"]], **a_property)

table_no +=1

table_two = add_table_to_page(ttwo, table_no, caption, section, page, " ")
glue('tablea12', table_two, display=True)

,Échantillon,Particules,% échantillon,% particules,Moyenne
Amphion,10,1'250,"0,05","0,03","125,00"
Anthy,6,685,"0,03","0,02","114,17"
Aubonne,8,757,"0,04","0,02","94,62"
Baby plage,11,1'288,"0,05","0,03","117,09"
Bouveret,11,2'114,"0,05","0,05","192,18"
Clarens,6,415,"0,03","0,01","69,17"
Crans,1,19,"0,00","0,00","19,00"
Cully,1,128,"0,00","0,00","128,00"
Excenevex,28,3'775,"0,13","0,10","134,82"
Gland,1,40,"0,00","0,00","40,00"


In [5]:
bar_plots = pd.melt(location_summary, id_vars="Plage", value_vars=["% échantillon", "% particules"])
fig, ax = plt.subplots()
sns.barplot(data=bar_plots, x="Plage", y="value", hue="variable", ax=ax)
ax.legend(loc="upper left")
ax.tick_params(axis="x", labelrotation=90, labelright=True, labelsize=12)
ax.set_ylabel('% du total')
ax.set_xlabel(" ")

glue('fig-A11', fig, display=False)
plt.close()

```{glue:figure} fig-A11
---
name: fig-A11
---
{glue:text}`blank_caption` 
```

In [6]:
attribute_columns = ["Plage", "orientation", "position", "substrat", "fréquentation", "situation", "distance"]
percent_of_samples = df1.loc[select_values]['échantillon'].sum()/location_summary['échantillon'].sum()
attributes_of_samples = work_data[work_data.Plage.isin(test_one)].drop_duplicates(["Plage"])

attribute_keys = [name_zones, name_substrate, name_frequentation, name_situation, name_distance]
caption = "Les plages et leurs attributs où le proportion du comptage est supérieur au proportion des échantillons."    

display_df = translate_columns(attributes_of_samples, attribute_columns[2:], attribute_keys)
display_df.set_index("Plage", inplace=True, drop=True)
display_df.index.name = None
table_three = display_df[attribute_columns[1:]].style.set_table_styles(table_css_styles)


table_no += 1


table_three = add_table_to_page(table_three, table_no, caption, section, page, " ")
glue('tablea13', table_three, display=True)

,Orientation,Position,Substrat,Fréquentation,Situation,Distance
Lugrin,NNE,ligne d'eau,Cailloux,Moyenne,Campagne,100 - 500 m
Port choiseul,NE,ligne d'eau,Graviers,Elevée,Urbain,100 - 500 m
Savonnière,NO,ligne d'eau,Cailloux,Elevée,Campagne,< 100 m
Grangettes,NO,ligne d'eau,Sables fins,Moyenne,Campagne,100 - 500 m
Pichette,SSE,ligne d'eau,Graviers,Moyenne,Campagne,< 100 m
Préverenges,SO,ligne d'eau,Sables fins,Elevée,Campagne,100 - 500 m
Bouveret,ONO,ligne d'eau,Sables fins,Elevée,Campagne,100 - 500 m


(micro-table-A1-4)=
## Compte moyen par échantillon et plage

In [7]:
summary_particuless = sample_totals.particules.describe()

summary_particuless_display =  translate_describe(summary_particuless.T, "particules", transpose=True).style.set_table_styles(table_css_styles)

table_no += 1
caption = "Valeurs récapitulatives des totaux des échantillons."

table_four = add_table_to_page(summary_particuless_display, table_no, caption, section, page, " ")
glue('tablea14', table_four, display=True)

,Min,25%,50%,75%,Max,Moyenne,Écart-type
Particules,"19,00","72,00","124,00","200,00","2'991,00","183,52","268,98"


In [8]:
fig, ax = plt.subplots(2,2, figsize=(7,7))

sns.scatterplot(data=sample_totals, x="échantillon", y="particules", ax=ax[0,0])
ax[0,0].tick_params(labelbottom=False, bottom=False)
ax[0,0].set_ylim(-1, 1500)
ax[0,0].set_title("Total par échantillon", loc="left")
capitalize_x_and_y_axis_labels(ax[0,0])
# capitalize_legend_components(ax[0,0])
# sns.move_legend(ax[0, 0], title=" ", loc='best')

sns.boxplot(data=sample_totals, y="particules", dodge=False, width=.9, ax=ax[0,1])
ax[0,1].set_ylim(-1, 1500)
ax[0,1].set_title("Boîte de Tukey", loc="left")
capitalize_x_and_y_axis_labels(ax[0,1])


sns.histplot(data=sample_totals, x="particules", ax=ax[1,0], stat="probability", kde=True)
ax[1,0].set_xlim(-1, 1500)
ax[1,0].yaxis.set_major_formatter('{x:.2f}')
ax[1,0].set_ylabel("Probabilité")
ax[1,0].set_title("Histogramme", loc="left")
capitalize_x_and_y_axis_labels(ax[1,0])

sns.ecdfplot(data=sample_totals, x="particules", ax=ax[1,1])
ax[1,1].set_xlim(-1, 1500)
ax[1,1].set_title("Fonction de répartition", loc="left")
capitalize_x_and_y_axis_labels(ax[1,1])
plt.tight_layout()
glue('fig-A12', fig, display=False)
plt.close()

```{glue:figure} fig-A12
---
name: fig-A12
---
{glue:text}`blank_caption` 
```

## Compte moyenne par position 

In [9]:
position_totals = work_data.groupby(["échantillon", "position"], as_index=False).agg({'particules': 'sum'})

column_display = {
    "count":"échantillons",
    "mean": "moyenne",
    "std": "écart type",
    "échantillon":"échantillon"
}

voi = "position"
vals = "particules"
groupbys = ['échantillon', voi]
labels = name_zones
position_summary = attribute_summary(position_totals, vals, voi, columns=column_display, labels=name_zones, as_type='float')
table_no += 1
caption = "Le résumé des totaux d'échantillons par position sur la plage."

table_five = add_table_to_page(position_summary, table_no, caption, section, page," ")
glue('tablea15', table_five, display=True)

,Échantillons,Moyenne,Écart type,Min,25%,50%,75%,Max
Ligne d'eau,111,"116,84","125,53","19,00","53,50","85,00","129,00","884,00"
Plage seche,102,"256,08","352,84","36,00","114,25","172,50","241,75","2'991,00"


In [10]:
fig, ax = plt.subplots(2,2, figsize=(7,7))
position_totals["position"] = position_totals.position.apply(lambda x: name_zones[x])
d_position = position_totals.groupby(["échantillon", "position"], as_index=False).particules.sum()
d_position["position"] = d_position["position"].apply(lambda x: x.capitalize())

sns.scatterplot(data=d_position, x="échantillon", y="particules", hue="position", ax=ax[0,0])
ax[0,0].tick_params(labelbottom=False, bottom=False)
ax[0,0].set_ylim(-1, 1000)
ax[0,0].set_title("Total par échantillon", loc="left")
ax[0,0].legend()
capitalize_x_and_y_axis_labels(ax[0,0])
sns.move_legend(ax[0, 0], title=" ", loc='best')


sns.boxplot(data=d_position, x="position", y="particules", hue="position", dodge=False, width=.9, ax=ax[0,1])
ax[0,1].set_ylim(-1, 1000)
ax[0,1].set_title("Boîte de Tukey", loc="left")
# ax[0,1].get_legend().remove()
ax[0,1].set_xlabel("")
# capitalize_x_tick_labels(ax[0,1])
capitalize_x_and_y_axis_labels(ax[0,1])

sns.histplot(data=d_position, x="particules", hue="position", ax=ax[1,0], multiple="stack", stat="probability", kde=True)
ax[1,0].set_xlim(-10, 1000)
ax[1,0].yaxis.set_major_formatter('{x:.2f}')
ax[1,0].set_title("Histogramme", loc="left")
ax[1,0].get_legend().remove()
capitalize_x_and_y_axis_labels(ax[1,0])


sns.ecdfplot(data=d_position, x="particules", hue="position", ax=ax[1,1])
ax[1,1].set_xlim(-1, 1000)
ax[1,1].set_title("Fonction de répartition", loc="left")
capitalize_x_and_y_axis_labels(ax[1,1])
sns.move_legend(ax[1, 1], title=" ", loc='best')


plt.tight_layout()
glue('fig-A13', fig, display=False)

plt.close()

```{glue:figure} fig-A13
---
name: fig-A13
---
{glue:text}`blank_caption` 
```

### Nombre moyen par forme et plage

In [11]:
forme_totals = work_data.groupby(["Plage","échantillon", "objet"], as_index=False).particules.sum()
df3 = forme_totals.copy()

average_of_sample_formes = df3.groupby(["objet"], as_index=True).agg({"particules":"mean"})
average_of_sample_formes.rename(columns={"particules":value_column}, inplace=True)

column_display = {
    "count":"échantillons",
    "mean": "moyenne",
    "std": "écart type",
    "échantillon":"échantillon"
}

voi = "objet"
vals = "particules"
groupbys = ['échantillon', voi]
labels = particle_groups
form_summary = attribute_summary(forme_totals, vals, voi, columns=column_display, labels=labels, as_type='float')
table_six = add_table_to_page(form_summary, table_no, caption, section, page, " ")
glue("table-six", table_six, display=True)

,Échantillons,Moyenne,Écart type,Min,25%,50%,75%,Max
Fibre,213,"140,42","212,82","17,00","58,00","98,00","156,00","2'593,00"
Particule rigide,213,"23,97","58,58","0,00","4,00","11,00","24,00","696,00"
Particule souple,213,"19,13","60,65","0,00","1,00","5,00","14,00","604,00"


In [12]:
# the average sample total by form of particle
forme_averages = df3.groupby(["Plage", "objet"], as_index=False).agg({"particules":"mean"})
forme_averages.rename(columns={"particules":value_column}, inplace=True)

forme_averages = forme_averages.pivot(index="Plage", columns="objet").droplevel(0, axis=1)
forme_averages.index.name = None
forme_averages.rename(columns=particle_groups, inplace=True)

# identifyt the values that exceed threshold
select_fdure_values = forme_averages["Particule rigide"] > average_of_sample_formes.loc["fdure", value_column]
select_souple_values = forme_averages["Particule souple"] > average_of_sample_formes.loc["souple", value_column]
select_fiber_vales = forme_averages["Fibre"] > average_of_sample_formes.loc["fibres", value_column]

# select the the values that exceed threshold
test_three = forme_averages.loc[select_fdure_values].index
test_four = forme_averages.loc[select_souple_values].index
test_five = forme_averages.loc[select_fiber_vales].index

caption = "Le nombre moyen par échantillon par plage et forme."

sa = forme_averages.style.set_table_styles(table_css_styles)

# highlight values that exceed table styles
sa = sa.set_properties(subset = pd.IndexSlice[test_three,["Particule rigide"]], **a_property)
sa = sa.set_properties(subset = pd.IndexSlice[test_four,["Particule souple"]], **a_property)
sa = sa.set_properties(subset = pd.IndexSlice[test_five,["Fibre"]], **a_property)
sa.columns.name = None

table_no += 1

table_seven = add_table_to_page(sa, table_no, caption, section, page, rule)
glue("table-seven", table_seven, display=True)

,Particule rigide,Fibre,Particule souple
Amphion,"7,90","110,00","7,10"
Anthy,"9,17","91,67","13,33"
Aubonne,"11,88","70,12","12,62"
Baby plage,"12,00","98,09","7,00"
Bouveret,"15,00","160,64","16,55"
Clarens,"6,33","57,50","5,33"
Crans,"2,00","17,00","0,00"
Cully,"11,00","117,00","0,00"
Excenevex,"19,39","97,14","18,29"
Gland,"8,00","26,00","6,00"


In [13]:
# def capitalize_x_tick_labels(an_ax):
#     print(an_ax.get_xticks())
#     an_ax.set_xticks(an_ax.get_xticks())
#     an_ax.set_xticklabels([x.get_text().capitalize() for x in an_ax.get_xticklabels()])

In [14]:
fig, ax = plt.subplots(2,2, figsize=(8,8))

df3["Objet"] = df3["objet"].apply(lambda x: particle_groups[x])

sns.scatterplot(data=df3, x="échantillon", y="particules", hue="Objet", ax=ax[0,0])
ax[0,0].tick_params(labelbottom=False, bottom=False)
ax[0,0].set_ylim(-1, 1000)
ax[0,0].set_title("Total par échantillon", loc="left")
capitalize_x_and_y_axis_labels(ax[0,0])
sns.move_legend(ax[0, 0], title=" ", loc='best')

sns.boxplot(data=df3, x="Objet", y="particules", hue="Objet", dodge=False, width=.9, ax=ax[0,1])
ax[0,1].set_ylim(-1, 1000)
ax[0,1].set_title("Boîte de Tukey", loc="left")
ax[0,1].set_xlabel("")
# ax[0,1].legend().remove()
capitalize_x_and_y_axis_labels(ax[0,1])
capitalize_x_tick_labels(ax[0,1])

sns.histplot(data=df3, x="particules", hue="Objet", ax=ax[1,0], multiple="stack", stat="probability", kde=True)
ax[1,0].set_xlim(-10, 600)
ax[1,0].yaxis.set_major_formatter('{x:.2f}')
ax[1,0].set_ylabel("Probabilité")
ax[1,0].set_title("Histogramme", loc="left")
ax[1,0].get_legend().remove()
capitalize_x_and_y_axis_labels(ax[1,0])

sns.ecdfplot(data=df3, x="particules", hue="Objet", ax=ax[1,1])
ax[1,1].set_xlim(-1, 600)
ax[1,1].set_title("Fonction de répartition", loc="left")
capitalize_x_and_y_axis_labels(ax[1,1])
sns.move_legend(ax[1, 1], title=" ", loc='best')

plt.tight_layout()

plt.tight_layout()
glue('fig-A14', fig, display=False)

plt.close()

```{glue:figure} fig-A14
---
name: fig-A14
---
{glue:text}`blank_caption` 
```

### Nombre moyen par forme et position

In [15]:
ldeau_form = work_data[work_data.position == 1].groupby(["échantillon", "Plage", "position", "objet"], as_index=False).particules.sum()

column_display = {
    "count":"échantillons",
    "mean": "moyenne",
    "std": "écart type",
    "échantillon":"échantillon"
}

voi = "objet"
vals = "particules"
groupbys = ['échantillon', voi]
labels = particle_groups
ldeau_summary = attribute_summary(ldeau_form, vals, voi, columns=column_display, labels=labels, as_type='float')

table_no += 1

caption = "<b>Ligne d'eau, </b> le nombre moyen de particules trouvées par échantillon selon la forme du particule."

table_eight = add_table_to_page(ldeau_summary, table_no, caption, section, page,' ')
glue("table-eight", table_eight, display=True)

,Échantillons,Moyenne,Écart type,Min,25%,50%,75%,Max
Fibre,111,"86,73","58,66","17,00","39,00","72,00","116,50","273,00"
Particule rigide,111,"12,24","16,40","0,00","2,00","7,00","14,00","112,00"
Particule souple,111,"17,86","81,22","0,00","0,00","1,00","3,50","604,00"


In [16]:
pseche_form = work_data[work_data.position == 2].groupby(["échantillon", "Plage", "position", "objet"], as_index=False).particules.sum()

column_display = {
    "count":"échantillons",
    "mean": "moyenne",
    "std": "écart type",
    "échantillon":"échantillon"
}

voi = "objet"
vals = "particules"
groupbys = ['échantillon', voi]
labels = particle_groups
psech_summary = attribute_summary(pseche_form, vals, voi, columns=column_display, labels=labels, as_type='float')

table_no += 1

caption = "<b>Plage seche, </b>le nombre moyen de particules trouvées par échantillon selon la forme du particule."

table_nine = add_table_to_page(psech_summary, table_no, caption, section, page, ' ')
glue("table-nine", table_nine, display=True)

,Échantillons,Moyenne,Écart type,Min,25%,50%,75%,Max
Fibre,102,"198,84","291,05","30,00","85,00","126,50","187,75","2'593,00"
Particule rigide,102,"36,73","81,21","1,00","9,25","16,00","31,75","696,00"
Particule souple,102,"20,51","23,06","0,00","6,00","11,50","27,75","153,00"


In [17]:
position_totals = work_data.groupby(["Plage","échantillon", "position", "objet"], as_index=False).particules.sum()
position_totals["position"] = position_totals.position.apply(lambda x: name_zones[x])
position_totals["objet"] = position_totals.objet.apply(lambda x: particle_groups[x])

df4 = position_totals.groupby(["Plage","position", "objet"], as_index=False).particules.mean()

df4 = df4.pivot(index="Plage", columns=["position", "objet"])
df4.index.name = None
position_columns = list(name_zones.values())
fragment_columns = list(particle_groups.values())

args_one = [("particules", position_columns[0], x) for x in fragment_columns]
args_two = [("particules", position_columns[1], x) for x in fragment_columns]

args_0 = select_a_set_of_values(df4, args_one[0], list(particle_groups.keys())[0], value_column, average_of_sample_formes).index
args_1 = select_a_set_of_values(df4, args_one[1], list(particle_groups.keys())[1], value_column, average_of_sample_formes).index
args_2  = select_a_set_of_values(df4, args_one[2], list(particle_groups.keys())[2], value_column, average_of_sample_formes).index

args_2_0 = select_a_set_of_values(df4, args_two[0], list(particle_groups.keys())[0], value_column, average_of_sample_formes).index
args_2_1 = select_a_set_of_values(df4, args_two[1], list(particle_groups.keys())[1], value_column, average_of_sample_formes).index
args_2_2 = select_a_set_of_values(df4, args_two[2], list(particle_groups.keys())[2], value_column, average_of_sample_formes).index

sax = df4.style.set_table_styles(table_css_styles)
  
sax = sax.set_properties(subset = pd.IndexSlice[args_2, args_one[2]],**a_property)
sax = sax.set_properties(subset = pd.IndexSlice[args_1,args_one[1]],**a_property)
sax = sax.set_properties(subset = pd.IndexSlice[args_0,args_one[0]], **a_property)
sax = sax.set_properties(subset = pd.IndexSlice[args_2_2, args_two[2]], **a_property)
sax = sax.set_properties(subset = pd.IndexSlice[args_2_1,args_two[1]],**a_property)
sax = sax.set_properties(subset = pd.IndexSlice[args_2_0,args_two[0]], **a_property)

table_no += 1

caption = "le nombre moyen de particules trouvées par plage selon la forme du particule et la position."

table_ten = add_table_to_page(sax, table_no, caption, section, page, ' ')
glue("table-ten", table_ten, display=True)

### Distributions



In [18]:
fig, ax = plt.subplots(1)

sns.boxplot(data=position_totals, x="position", y="particules", hue="objet", ax=ax)

ax.set_xlabel("")
capitalize_x_and_y_axis_labels(ax)
capitalize_x_tick_labels(ax)
capitalize_legend_components(ax)

plt.tight_layout()
glue('fig-A15', fig, display=False)

plt.close()

```{glue:figure} fig-A15
---
name: fig-A15
---
{glue:text}`blank_caption` 
```

In [19]:
position_totals = work_data.groupby(["Plage","échantillon", "position"], as_index=False).particules.sum()
position_totals["position"] = position_totals.position.apply(lambda x: name_zones[x])

fig, ax = plt.subplots(1, figsize=(8,5))



ax.set_xlabel("")
ax.tick_params(axis="x", labelrotation=90)
capitalize_x_and_y_axis_labels(ax)
capitalize_legend_components(ax)

plt.tight_layout()
glue('fig-A16', fig, display=False)

plt.close()

```{glue:figure} fig-A16
---
name: fig-A16
---
{glue:text}`blank_caption` 
```

In [20]:
%watermark --iversions -b -r

Git repo: https://github.com/hammerdirt-analyst/plastock.git

Git branch: feb5

matplotlib: 3.8.2
pandas    : 2.0.3
seaborn   : 0.13.1
numpy     : 1.26.3

